In [1]:
import logging
import pickle
import pandas as pd
from sklearn import metrics
from sklearn.metrics import f1_score
import keras
from keras.models import Model, Sequential
from keras.layers import *
from keras.preprocessing.text import one_hot
from keras.optimizers import Adam
from tqdm import tnrange, tqdm_notebook
from keras.layers.advanced_activations import LeakyReLU
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from sklearn.utils import class_weight
from keras.layers import Concatenate, Dense, LSTM, Input, Activation, concatenate

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
import tensorflow as tf
from keras.backend.tensorflow_backend import set_session
config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.3
set_session(tf.Session(config=config))

In [3]:
MODEL_PATH = "../models/"
DATA_PATH = "../data/"

In [4]:
train = pd.read_csv(DATA_PATH + "preprocessed/train_ling.csv")

In [5]:
targets = train[['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']].values

In [6]:
meta_features = ['count_sent', 'count_word', 'count_unique_word', 'count_letters',
       'count_punctuations', 'count_words_upper', 'count_words_title',
       'count_stopwords', 'mean_word_len', 'word_unique_percent',
       'punct_percent', 'count_swear_words']

In [7]:
train.head()

,Unnamed: 0,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate,count_sent,...,count_unique_word,count_letters,count_punctuations,count_words_upper,count_words_title,count_stopwords,mean_word_len,word_unique_percent,punct_percent,count_swear_words
0,0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0,2,...,41,264,10,2,11,16,5.162791,95.348837,23.255814,0
1,1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0,1,...,17,112,12,1,3,2,5.588235,100.000000,70.588235,0
2,2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0,1,...,39,233,6,0,2,19,4.571429,92.857143,14.285714,0
3,3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0,5,...,82,622,21,5,7,55,4.486726,72.566372,18.584071,0
4,4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0,1,...,13,67,5,0,2,5,4.230769,100.000000,38.461538,0


In [8]:
def build_input_data(sentences, labels, vocabulary):
    x = np.array([[embeddings_index[vocabulary_inv[vocabulary['word']]] if word in vocabulary.keys() else len(vocabulary) - 1 for word in sentence] for sentence in sentences])
    y = np.array(labels)
    return [x, y]

In [9]:
train_text = train['comment_text']

In [10]:
tokenizer = Tokenizer(num_words=18400)
tokenizer.fit_on_texts(train_text)
sequences = tokenizer.texts_to_sequences(train_text)

In [11]:
vocabulary = tokenizer.word_index
vocabulary_inv = {v:k for k, v in vocabulary.items()}
embeddings_index = {}
EMBEDDING_DIM = 100
f = open("../../../embeddings/glove.6B." + str(EMBEDDING_DIM) + "d.txt")
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

In [12]:
print('Found %s word vectors.' % len(embeddings_index))
embedding_matrix = np.zeros((len(vocabulary) + 1, EMBEDDING_DIM))
embedding_matrix[-1] = np.random.rand(EMBEDDING_DIM) # oov-vector
for word, i in vocabulary.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i - 1] = embedding_vector
embedding_layer = Embedding(embedding_matrix.shape[0],
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=200, trainable = False)

Found 400000 word vectors.


In [13]:
from sklearn.model_selection import train_test_split

In [14]:
x_train, x_dev, y_train, y_dev = train_test_split(train, targets, test_size=0.3, random_state=42)

In [15]:
max_length = 200

x_train_texts = tokenizer.texts_to_sequences(x_train['comment_text'])
x_train_texts = pad_sequences(x_train_texts, maxlen=max_length, padding='post')

x_dev_texts = tokenizer.texts_to_sequences(x_dev['comment_text'])
x_dev_texts = pad_sequences(x_dev_texts, maxlen=max_length, padding='post')

In [34]:
sequence_input = Input(shape=(max_length,), dtype='float32')
metadata_input = Input(shape=(len(meta_features),), dtype='float32')
embedded_sequences = embedding_layer(sequence_input)
lstm = keras.layers.CuDNNLSTM(10, return_sequences=False)(embedded_sequences)
concatenated_data = Concatenate(axis=1)([lstm, metadata_input])
reshape = keras.layers.Reshape((-1, 1))(concatenated_data)
lstm_2 = keras.layers.CuDNNLSTM(10, return_sequences=False)(reshape)
reshape_2 = keras.layers.Reshape((-1, 1))(lstm_2)
cnn = keras.layers.Conv1D(filters = 100, kernel_size = 2, activation = 'relu')(reshape_2)
pool = keras.layers.MaxPooling1D(pool_size=2)(cnn)
flatten = keras.layers.Flatten()(pool)
dense_1 = Dense(50, activation='relu')(flatten)
output_1 = Dense(units=1, activation='sigmoid', name = 'output_1')(dense_1)
output_2 = Dense(units=1, activation='sigmoid', name = 'output_2')(dense_1)
output_3 = Dense(units=1, activation='sigmoid', name = 'output_3')(dense_1)
output_4 = Dense(units=1, activation='sigmoid', name = 'output_4')(dense_1)
output_5 = Dense(units=1, activation='sigmoid', name = 'output_5')(dense_1)
output_6 = Dense(units=1, activation='sigmoid', name = 'output_6')(dense_1)
model = Model(inputs=[sequence_input,metadata_input], outputs=[output_1, output_2, output_3, output_4, output_5, output_6])
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['acc'])

model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_15 (InputLayer)           (None, 200)          0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 200, 100)     21033800    input_15[0][0]                   
__________________________________________________________________________________________________
cu_dnnlstm_14 (CuDNNLSTM)       (None, 10)           4480        embedding_1[7][0]                
__________________________________________________________________________________________________
input_16 (InputLayer)           (None, 12)           0                                            
__________________________________________________________________________________________________
concatenat

In [20]:
separate_targets_train = [y_train[:, i] for i in range(0, y_train.shape[1])]

In [21]:
separate_targets_dev = [y_dev[:, i] for i in range(0, y_dev.shape[1])]

In [22]:
separate_class_weights = [class_weight.compute_class_weight('balanced', np.unique(separate_targets_train[i]),separate_targets_train[i]) for i in range(0, len(separate_targets_train))]

In [23]:
separate_class_weights_dict = [{0:x[0], 1: x[1]} for x in separate_class_weights]

In [24]:
separate_class_weights_dict

[{0: 0.5530365294542862, 1: 5.213732262882749},
 {0: 0.5050140157337915, 1: 50.360234445446345},
 {0: 0.527848137156683, 1: 9.477261157305277},
 {0: 0.5015356017134083, 1: 163.30263157894737},
 {0: 0.5258353654517893, 1: 10.176658163265307},
 {0: 0.5043937286635478, 1: 57.39928057553957}]

In [25]:
separate_class_weights

[array([0.55303653, 5.21373226]),
 array([ 0.50501402, 50.36023445]),
 array([0.52784814, 9.47726116]),
 array([  0.5015356 , 163.30263158]),
 array([ 0.52583537, 10.17665816]),
 array([ 0.50439373, 57.39928058])]

In [26]:
multiple_class_weights = {'output_' + str(i + 1): x for i, x in enumerate(separate_class_weights_dict)}

In [27]:
separate_targets_train_ = [y_train[i, :] for i in range(0, y_train.shape[0])]
separate_targets_dev_ = [y_dev[i, :] for i in range(0, y_dev.shape[0])]

In [28]:
len(x_train_texts)

111699

In [35]:
from keras.callbacks import ModelCheckpoint
filepath="../models/contextual_lstm_cnn-{epoch:02d}-{val_loss:.2f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

In [36]:
model.fit([x_train_texts, x_train[meta_features]], separate_targets_train, validation_data=([x_dev_texts, x_dev[meta_features]], separate_targets_dev),
          epochs=20, batch_size=20, class_weight = multiple_class_weights, callbacks=callbacks_list)

Train on 111699 samples, validate on 47872 samples
Epoch 1/20
111699/111699 [==============================] - 204s 2ms/step - loss: 2.4346 - output_1_loss: 0.4836 - output_2_loss: 0.2917 - output_3_loss: 0.3790 - output_4_loss: 0.4431 - output_5_loss: 0.4269 - output_6_loss: 0.4103 - output_1_acc: 0.8540 - output_2_acc: 0.9062 - output_3_acc: 0.8842 - output_4_acc: 0.8830 - output_5_acc: 0.8953 - output_6_acc: 0.8780 - val_loss: 2.2089 - val_output_1_loss: 0.4729 - val_output_2_loss: 0.2680 - val_output_3_loss: 0.3090 - val_output_4_loss: 0.3481 - val_output_5_loss: 0.4133 - val_output_6_loss: 0.3976 - val_output_1_acc: 0.8785 - val_output_2_acc: 0.9243 - val_output_3_acc: 0.9294 - val_output_4_acc: 0.9134 - val_output_5_acc: 0.9173 - val_output_6_acc: 0.9078

Epoch 00001: val_loss improved from -inf to 2.20889, saving model to ../models/contextual_lstm_cnn-01-2.21.hdf5
Epoch 2/20
111699/111699 [==============================] - 201s 2ms/step - loss: 2.2977 - output_1_loss: 0.4645 - o

111699/111699 [==============================] - 221s 2ms/step - loss: 2.0605 - output_1_loss: 0.4273 - output_2_loss: 0.2201 - output_3_loss: 0.3308 - output_4_loss: 0.3484 - output_5_loss: 0.3632 - output_6_loss: 0.3707 - output_1_acc: 0.8673 - output_2_acc: 0.9301 - output_3_acc: 0.9148 - output_4_acc: 0.8680 - output_5_acc: 0.8885 - output_6_acc: 0.8906 - val_loss: 1.4786 - val_output_1_loss: 0.3208 - val_output_2_loss: 0.1541 - val_output_3_loss: 0.2467 - val_output_4_loss: 0.2528 - val_output_5_loss: 0.2550 - val_output_6_loss: 0.2492 - val_output_1_acc: 0.8980 - val_output_2_acc: 0.9457 - val_output_3_acc: 0.9276 - val_output_4_acc: 0.9064 - val_output_5_acc: 0.9189 - val_output_6_acc: 0.9164 - output_5 - ETA: 2:44 - loss: 2.0570 - output_1_loss: 0.4212 - output_2_loss: 0.2089 - output_3_loss: 0.3316 - output_4_loss: 0.3460 - output_5_loss: 0.3721 - output_6_loss: 0.3771 - output_1_acc: 0.8677 - output_2_acc: 0.9289 - output_3_acc: 0.9124 - output_4_acc: 0.883 - ETA: 2:40 - loss

111699/111699 [==============================] - 225s 2ms/step - loss: 1.8275 - output_1_loss: 0.3948 - output_2_loss: 0.1818 - output_3_loss: 0.3076 - output_4_loss: 0.2754 - output_5_loss: 0.3297 - output_6_loss: 0.3383 - output_1_acc: 0.8725 - output_2_acc: 0.9155 - output_3_acc: 0.9014 - output_4_acc: 0.8821 - output_5_acc: 0.8867 - output_6_acc: 0.8748 - val_loss: 2.1190 - val_output_1_loss: 0.4351 - val_output_2_loss: 0.2182 - val_output_3_loss: 0.3322 - val_output_4_loss: 0.3909 - val_output_5_loss: 0.3719 - val_output_6_loss: 0.3707 - val_output_1_acc: 0.8399 - val_output_2_acc: 0.9137 - val_output_3_acc: 0.8850 - val_output_4_acc: 0.8278 - val_output_5_acc: 0.8600 - val_output_6_acc: 0.85563929 - output_2_loss: 0.1853 - output_3_loss: 0.3039 - output_4_loss: 0.2558 - output_5_loss: 0.3266 - output_6_loss: 0.3381 - output_1_acc: 0.8758 - output_2_acc:  - ETA: 57s - loss: 1.8029 - output_1_loss: 0.3922 - output_2_loss: 0.1857 - output_3_loss: 0.3035 - output_4_loss: 0.2532 - out

111699/111699 [==============================] - 219s 2ms/step - loss: 1.6380 - output_1_loss: 0.3756 - output_2_loss: 0.1602 - output_3_loss: 0.2884 - output_4_loss: 0.2079 - output_5_loss: 0.3092 - output_6_loss: 0.2967 - output_1_acc: 0.8679 - output_2_acc: 0.9241 - output_3_acc: 0.9050 - output_4_acc: 0.8885 - output_5_acc: 0.8885 - output_6_acc: 0.8728 - val_loss: 1.8605 - val_output_1_loss: 0.3795 - val_output_2_loss: 0.2228 - val_output_3_loss: 0.3033 - val_output_4_loss: 0.2904 - val_output_5_loss: 0.3089 - val_output_6_loss: 0.3555 - val_output_1_acc: 0.8633 - val_output_2_acc: 0.9113 - val_output_3_acc: 0.8967 - val_output_4_acc: 0.8710 - val_output_5_acc: 0.8875 - val_output_6_acc: 0.8580 - ETA: 1:49 - loss: 1.5854 - output_1_loss: 0.3722 - output_2_loss: 0.1458 - output_3_loss: 0.2823 - output_4_loss: 0.1725 - output_5_loss: 0.3135 - output_6_loss: 0.2991 - output_1_acc: 0.8720 - output_2_acc: 0.9258 - output_3_acc: 0.9059 - output_4_acc: 0.8978 - output_5_a - ETA: 1:46 - l

In [37]:
pred_dev = model.predict([x_dev_texts, x_dev[meta_features]])

In [40]:
pred_dev = np.hstack(pred_dev)

In [41]:
def mean_roc_auc(y_true, y_pred):
    roc_auc_scores = []
    for i in range(0, y_true.shape[1]):
        roc_auc_scores.append(metrics.roc_auc_score(y_true[:, i], y_pred[:, i]))
    print(roc_auc_scores)
    return np.mean(roc_auc_scores)

In [42]:
mean_roc_auc(y_dev, pred_dev)

[0.9086792589520657, 0.9740130048308272, 0.9455602742888023, 0.9407368590975856, 0.9373856369428853, 0.9382553888732748]


0.9407717371642401

In [43]:
model_json = model.to_json()
with open(MODEL_PATH + "keras_contextual_lstm_cnn_classification_model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights(MODEL_PATH + "keras_contextual_lstm_cnn_classification_model.h5")
print("Saved model to disk")

Saved model to disk


In [44]:
model_name = 'keras_contextual_lstm_cnn_classification_model'

In [45]:
test = pd.read_csv(DATA_PATH + "preprocessed/test_ling.csv")

x_dev_texts = tokenizer.texts_to_sequences(test['comment_text'])
x_dev_texts = pad_sequences(x_dev_texts, maxlen=max_length, padding='post')

pred = model.predict([x_dev_texts, test[meta_features]])

list_classes = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]
sample_submission = pd.read_csv('../submissions/sample_submission.csv')
sample_submission[list_classes] = np.hstack(pred)
sample_submission.to_csv("../submissions/" + model_name + ".csv", index=False)